In [11]:
# 구글 드라이브 연결

from google.colab import drive
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/ch6'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 작업 디렉토리: /content/drive/MyDrive/ch6


In [12]:
!apt-get update && apt-get install -y gcc afl
!pip install python-afl

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,596 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,645 kB]
Hit:14 https://ppa.launchpadcontent.net/ubun

In [13]:
!afl-gcc -o bof_afl bof.c

afl-cc++4.00c by Michal Zalewski, Laszlo Szekeres, Marc Heuse - mode: GCC-GCC
[!] WARNING: You are using outdated instrumentation, install LLVM and/or gcc-plugin and use afl-clang-fast/afl-clang-lto/afl-gcc-fast instead!
bof.c: In function ‘main’:
bof.c:75:9: warning: ignoring return value of ‘fgets’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   75 |         fgets(input, sizeof(input), stdin);
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
afl-as++4.00c by Michal Zalewski
[+] Instrumented 24 locations (64-bit, non-hardened mode, ratio 100%).


In [14]:
# AFL용 입력 디렉터리 및 초기 샘플 입력값 생성
!mkdir -p input
with open("input/sample.txt", "w") as f:
    f.write("AAAAAAAAAAAAAAAAAAAAA")

In [15]:
with open("bof_afl_fuzz.py", "r") as f:
  file_contents = f.read()
  print(file_contents)

import afl
import subprocess
import sys

# AFL을 초기화하고 퍼징을 시작했는지 확인
if afl.init():
    print("AFL is now fuzzing...")

# AFL 퍼징할 실행 파일 경로
target_binary = "./bof_afl"  # AFL로 컴파일된 C 프로그램 경로

# 프로그램 실행 및 입력값 전달 함수
def fuzz_input(input_data):
    # C 프로그램을 subprocess로 실행하고, 입력 데이터를 전달하여 결과를 받음
    process = subprocess.Popen([target_binary], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    try:
        # 입력 데이터를 프로세스에 전달하고 결과를 기다림
        stdout, stderr = process.communicate(input=input_data, timeout=2)
    except subprocess.TimeoutExpired:
        process.kill()
        stdout, stderr = process.communicate()
        print("Program timeout occurred.")

    # 프로그램의 출력 및 종료 코드 반환
    return stdout, stderr, process.returncode

# AFL이 직접적으로 Python 스크립트를 퍼징하는 경우
# 이 반복문을 통해 지속적으로 입력을 받아 처리할 수 있도록 구성
while afl.loop():
    # 표준 입력으로부터 입력값을 읽음 (AFL이 전달하는 입력)
    input_data = sys.stdin.read()
    
    # 입력 데이터를 C 프로그램에 전달하여 실행
    stdout, stderr, exit_code = fuzz_input

In [22]:
!AFL_SKIP_BIN_CHECK=1 AFL_I_DONT_CARE_ABOUT_MISSING_CRASHES=1 afl-fuzz -n -i input -o output -t 10000+ -- ./bof_afl

[!] WARNING: Potentially mistyped AFL environment variable: USE_AUTH_EPHEM=1, did you mean AFL_USE_AUTH_EPHEM=1?
afl-fuzz++4.00c based on afl by Michal Zalewski and a large online community
[+] afl++ is maintained by Marc "van Hauser" Heuse, Heiko "hexcoder" Eißfeldt, Andrea Fioraldi and Dominik Maier
[+] afl++ is open source, get it at https://github.com/AFLplusplus/AFLplusplus
[+] NOTE: This is v3.x which changes defaults and behaviours - see README.md
[*] Getting to work...
[+] Using exponential power schedule (FAST)
[+] Enabled testcache with 50 MB
[*] Checking core_pattern...

[-] Hmm, your system is configured to send core dump notifications to an
    external utility. This will cause issues: there will be an extended delay
    between stumbling upon a crash and having this information relayed to the
    fuzzer via the standard waitpid() API.
    If you're just testing, set 'AFL_I_DONT_CARE_ABOUT_MISSING_CRASHES=1'.

    To avoid having crashes misinterpreted as timeouts, please 

In [10]:
!echo "AAAAAAAAAAAAAAAAAAA" | ./bof_afl
!which python3

/bin/bash: line 1: ./bof_afl: Permission denied
/usr/bin/python3


In [ ]:
!AFL_SKIP_BIN_CHECK=1 AFL_I_DONT_CARE_ABOUT_MISSING_CRASHES=1 afl-fuzz -n -i input -o output -- /usr/bin/python3 bof_afl_fuzz.py

[+] Loaded environment variable AFL_I_DONT_CARE_ABOUT_MISSING_CRASHES with value 1
[+] Loaded environment variable AFL_SKIP_BIN_CHECK with value 1
[!] WARNING: Potentially mistyped AFL environment variable: USE_AUTH_EPHEM=1, did you mean AFL_USE_AUTH_EPHEM=1?
afl-fuzz++4.00c based on afl by Michal Zalewski and a large online community
[+] afl++ is maintained by Marc "van Hauser" Heuse, Heiko "hexcoder" Eißfeldt, Andrea Fioraldi and Dominik Maier
[+] afl++ is open source, get it at https://github.com/AFLplusplus/AFLplusplus
[+] NOTE: This is v3.x which changes defaults and behaviours - see README.md
[!] WARNING: Target binary called without a prefixed path, make sure you are fuzzing the right binary: python3
[*] Getting to work...
[+] Using exponential power schedule (FAST)
[+] Enabled testcache with 50 MB
[*] Checking core_pattern...

[-] Hmm, your system is configured to send core dump notifications to an
    external utility. This will cause issues: there will be an extended delay
  

In [ ]:
!ls output/crashes/